In [1]:
import pandas as pd
import json
import mapply
import re

mapply.init(
    n_workers=-1,
    chunk_size=1000,
    max_chunks_per_worker=0,
    progressbar=True
)

/home/user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print('hello world')

hello world


In [5]:
!pwd

/home/user/itemdataquality/itemdataqualityrepo


In [4]:
%cd ..

/home/user/itemdataquality/itemdataqualityrepo


/home/user/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
%cd ..
%cd data

/home/user/itemdataquality
/home/user/itemdataquality/data


In [8]:
!curl https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Prime_Pantry.json.gz --output ../data/meta_Prime_Pantry.json.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5157k  100 5157k    0     0  5743k      0 --:--:-- --:--:-- --:--:-- 5737k


In [9]:
!gzip -d meta_Prime_Pantry.json.gz

In [12]:
with open('meta_Prime_Pantry.json') as f:
    data = [json.loads(line) for line in f]

In [13]:
data[0]

{'category': [],
 'tech1': '',
 'description': ['Sink your sweet tooth into MILK DUDS Candya delicious blend of smooth milk chocolate and chewy caramel. These bite-sized candies are perfect for popping whether youre at home, on the go, or watching movies with friends. Stock up and always have a mouthwatering treat within reach.'],
 'fit': '',
 'title': "HERSHEY'S Milk Duds Candy, 5 Ounce(Halloween Candy)",
 'also_buy': ['B019KE37WO', 'B007NQSWEU'],
 'tech2': '',
 'brand': 'Milk Duds',
 'feature': [],
 'rank': [],
 'also_view': [],
 'details': {'ASIN: ': 'B00005BPJO', 'Item model number:': '1070002152'},
 'main_cat': '<img src="https://m.media-amazon.com/images/G/01/pantry/subnav/pantry-subnav-logo._CB474181323_.png" class="nav-categ-image" alt="Prime Pantry"/>',
 'similar_item': '',
 'date': '',
 'price': '$5.00',
 'asin': 'B00005BPJO',
 'imageURL': ['https://images-na.ssl-images-amazon.com/images/I/51v07pI0ruL._SS40_.jpg',
  'https://images-na.ssl-images-amazon.com/images/I/31MiXt54ZZ

In [104]:
df = pd.read_json('meta_Luxury_Beauty.json',lines=True)

In [105]:
df = df.drop(['category','tech1','fit','tech2','similar_item','date','also_view','also_buy','imageURL','imageURLHighRes'],axis=1)

In [106]:
df['description'] = df['description'].mapply(lambda x: '; '.join(x) if x is not None and isinstance(x, list) else x)

/home/user/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 12/12 [00:00<00:00, 23.35it/s]


In [107]:
def extract_dims(x):
    try:
        dims=x['\n    Product Dimensions: \n    '].split(';')[0].strip()
        return dims
    except:
        return None

In [108]:
def extract_weight(x):
    try:
        weight=x['\n    Product Dimensions: \n    '].split(';')[1].strip()
        return weight
    except:
        return None

In [109]:
df['product_dims'] = df['details'].mapply(lambda x: extract_dims(x))
df['product_weight'] = df['details'].mapply(lambda x: extract_weight(x))

/home/user/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


100%|██████████| 12/12 [00:00<00:00, 14.31it/s]
/home/user/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 12/12 [00:00<00:00, 14.96it/s]


In [114]:
df['rank'] = df['rank'].mapply(lambda x: re.sub('[^a-zA-Z0-9&, ]|amp', '', x) if x is not None and isinstance(x, str) else x)

/home/user/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 12/12 [00:00<00:00, 102.45it/s]


In [112]:
df = df[['asin','main_cat','title','description', 'brand',
        'price','product_dims', 'product_weight','details','feature','rank']]

In [116]:
df.to_csv('lux_beauty.csv',index=False)

In [117]:
df

,asin,main_cat,title,description,brand,price,product_dims,product_weight,details,feature,rank
0,B00004U9V2,Luxury Beauty,Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,After a long day of handling thorny situations...,,$30.00,2.2 x 2.2 x 7 inches,8.8 ounces,{' Product Dimensions: ': '2.2 x 2.2 ...,[],"4,324 in Beauty & Personal Care"
1,B0000531EN,Luxury Beauty,AHAVA Bath Salts,If you haven't experienced the pleasures of ba...,,,3 x 3.5 x 6 inches,2.2 pounds,{' Product Dimensions: ': '3 x 3.5 x ...,[],"1,633,549 in Beauty & Personal Care"
2,B0000532JH,Luxury Beauty,"AHAVA Dead Sea Mineral Mud, 8.5 oz, Pack of 4","Rich, black mineral mud, harvested from the ba...",,,5.1 x 3 x 5.5 inches,2.48 pounds,{' Product Dimensions: ': '5.1 x 3 x ...,[],"1,806,710 in Beauty & Personal Care"
3,B00005A77F,Luxury Beauty,"Crabtree &amp; Evelyn Hand Soap, Gardeners, 10...",This liquid soap with convenient pump dispense...,,$15.99,2.6 x 2.6 x 6.7 inches,1.5 pounds,{' Product Dimensions: ': '2.6 x 2.6 ...,[],[]
4,B00005NDTD,Luxury Beauty,Soy Milk Hand Crme,Remember why you love your favorite blanket? T...,,$18.00,7.2 x 2.2 x 7.2 inches,4 ounces,{' Product Dimensions: ': '7.2 x 2.2 ...,[],"42,464 in Beauty & Personal Care"
...,...,...,...,...,...,...,...,...,...,...,...
12294,B01HIQIEYC,Luxury Beauty,"CND Shellac Power Polish, Patina Buckle",; CND Craft Culture Collection: Patina Buckle;...,,$15.95,None,None,"{' Item Weight: ': '0.48 ounces', 'Sh...",[],"88,740 in Beauty & Personal Care"
12295,B01HIQHQU0,Luxury Beauty,CND Shellac power polish denim patch,CND Shellac was designed to be used as a syste...,,$15.95,None,None,"{'Shipping Weight:': '1.4 ounces (', 'ASIN:': ...",[],"122,331 in Beauty & Personal Care"
12296,B01HIQEOLO,Luxury Beauty,"CND Shellac, Leather Satchel",CND Shellac was designed to be used as a syste...,,$15.95,None,None,"{'Shipping Weight:': '1.4 ounces (', 'Domestic...",[],"168,028 in Beauty & Personal Care"
12297,B01HJ2UY0W,Luxury Beauty,"Juicy Couture I Love Juicy Couture, 1.7 fl. Oz...",The I AM JUICY COUTURE girl is once again taki...,,$76.00,3.3 x 2.7 x 4.6 inches,None,{' Product Dimensions: ': '3.3 x 2.7 ...,[],"490,755 in Beauty & Personal Care"
